In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
# from six.moves import urlli
import tensorflow.compat.v2.feature_column as fc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

test = pd.read_csv("data/test.csv")
train =pd.read_csv("data/train.csv")
#cleaning data  
SPECIES = ['Closed', 'Open']
dftest = test.drop(columns=["CustomerId", "Surname","Balance","id"])
dftrain = train.drop(columns=["CustomerId", "Surname","Balance","id",)

dftrain['HasCrCard']=pd.Series(np.where(dftrain.HasCrCard.values == 0.0, 0, 1),dftrain.index) #setting these to an int without a decimal
dftest['HasCrCard']=pd.Series(np.where(dftest.HasCrCard.values == 0.0, 0, 1),dftest.index)
dftrain['IsActiveMember']=pd.Series(np.where(dftrain.IsActiveMember.values == 0.0, 0, 1),dftrain.index) #setting these to an int without a decimal
dftest['IsActiveMember']=pd.Series(np.where(dftest.IsActiveMember.values == 0.0, 0, 1),dftest.index)

y = dftrain.pop('Exited')
X_train, X_eval, y_train, y_eval = train_test_split(dftrain, y, test_size=0.15, random_state=1)

X_eval.head()



,CreditScore,Geography,Gender,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
51719,659,Germany,Male,48.0,4,2,0,1,134094.02
77286,756,Germany,Female,54.0,4,4,1,0,7166.71
43625,636,France,Male,46.0,5,1,1,0,48986.18
26151,613,France,Male,47.0,3,2,1,1,17802.42
60732,711,Germany,Female,51.0,4,1,0,0,172618.52


In [ ]:
CATEGORICAL_COLUMNS = ['Geography', 'Gender', 'Tenure', 'NumOfProducts','HasCrCard','IsActiveMember']
NUMERIC_COLUMNS = ['Age','CreditScore', 'EstimatedSalary'] #wasn't 100% sure on which was which
gender_column = tf.feature_column.categorical_column_with_vocabulary_list(key="Gender", vocabulary_list=["Male", "Female"], default_value=0)
geography_column = tf.feature_column.categorical_column_with_vocabulary_list(key="Geography", vocabulary_list=["France", "Spain", "Germany"], default_value=0)

my_feature_columns = [
    tf.feature_column.indicator_column(geography_column),
    tf.feature_column.indicator_column(gender_column),
    tf.feature_column.numeric_column(key='Tenure'),
    tf.feature_column.numeric_column(key='NumOfProducts'),
    tf.feature_column.numeric_column(key='HasCrCard'),
    tf.feature_column.numeric_column(key='IsActiveMember'),
    tf.feature_column.numeric_column(key='CreditScore'),
    tf.feature_column.numeric_column(key='EstimatedSalary'),
    tf.feature_column.numeric_column(key='Age')
]
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

In [12]:

def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 2 classes.
    n_classes=2)
classifier.train(
    input_fn=lambda: input_fn(dftrain, y_train, training=True),
    steps=5000)




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\zslus\\AppData\\Local\\Temp\\tmpgvuatfyg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


ValueError: Items of feature_columns must be a <class 'tensorflow.python.feature_column.feature_column_v2.DenseColumn'>. You can wrap a categorical column with an embedding_column or indicator_column. Given: VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(X_eval, y_eval, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

NOT USING THIS YET. want to use this to print the results from test.

In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))